In [1]:
import tensorflow as teras

import pandas as pandinha

from numpy import array as array_numerico

In [2]:
nome_das_colunas = ['LV-ActivePower-kW', 'Wind-Speed-m/s', 'Theoretical-Power-Curve-KWh', 'Wind-Direction', 'potenciado']

funcionamento = ['Fora do Limite Aceitável', 'Dentro do Limite Aceitável']

resultados = []

In [3]:
arquivo = pandinha.read_csv(teras.keras.utils.get_file(
    'raw_turbina_scala.csv',
    'https://raw.githubusercontent.com/vhsmdev/analytics-turbina-eolica/main/data/raw/raw_turbina_scala.csv'))

potenciado = []

for p, potencia in enumerate(list(arquivo['LV ActivePower (kW)'])):
  if potencia >= (arquivo.loc[p, 'Theoretical_Power_Curve (KWh)'] * 0.95) and potencia <= (arquivo.loc[p, 'Theoretical_Power_Curve (KWh)'] * 1.05):
    potenciado.append(1)

  else:
    potenciado.append(0)

arquivo['potenciado'] = potenciado

arquivo = pandinha.concat([arquivo, arquivo.loc[(arquivo['LV ActivePower (kW)'] == 0.0) |
                                                  (arquivo['Wind Speed (m/s)'] == 0.0) |
                                                  (arquivo['Theoretical_Power_Curve (KWh)'] == 0.0) |
                                                  (arquivo['Wind Direction (°)'] == 0.0)]])

arquivo = arquivo.drop_duplicates(keep = False).drop('Date/Time', axis = 1)

arquivo.columns = nome_das_colunas

treino = arquivo[9821:]

prova = arquivo[:9821]

for embaralhamento in range(1000):
  treino = treino.sample(frac = 1).reset_index(drop = True)

  prova = prova.sample(frac = 1).reset_index(drop = True)

treino_y = treino.pop('potenciado')

prova_y = prova.pop('potenciado')

3921320/3921320 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
classificador = teras.keras.Sequential([teras.keras.Input(shape = (4, )),
                                        teras.keras.layers.Dense(24, activation='relu'),
                                        teras.keras.layers.Dense(12, activation='relu'),
                                        teras.keras.layers.Dense(2, activation='softmax')])

classificador.compile(optimizer = 'adam',
                      loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])

treinamento = classificador.fit(treino, treino_y,
                                epochs = 20,
                                validation_data = (prova, prova_y))

print(f'Precisão do Modelo: {treinamento.history["val_accuracy"][19] * 100 :.2f}%')

Epoch 1/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8237 - loss: 15.2322 - val_accuracy: 0.9367 - val_loss: 0.2162
Epoch 2/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9493 - loss: 0.1933 - val_accuracy: 0.9342 - val_loss: 0.4803
Epoch 3/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9505 - loss: 0.2758 - val_accuracy: 0.9608 - val_loss: 0.1069
Epoch 4/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9671 - loss: 0.1472 - val_accuracy: 0.9543 - val_loss: 0.1675
Epoch 5/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9691 - loss: 0.1655 - val_accuracy: 0.9455 - val_loss: 0.2232
Epoch 6/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9550 - loss: 0.3072 - val_accuracy: 0.8601 - val_loss: 1.7704
Epoch 7/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9647 - loss: 0.1716 - val_accuracy: 0.9867 - val_loss: 0.0397
Epoch 8/20
921/921 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9751 - loss: 0.1050 - val_accuracy: 0

In [6]:
def entrada(recursos):
  return teras.data.Dataset.from_tensor_slices(array_numerico(recursos)).batch(256)

In [7]:
recursado = ['LV-ActivePower-kW', 'Wind-Speed-m/s', 'Theoretical-Power-Curve-KWh', 'Wind-Direction']
prever = []

380.7 5.4 458.2 11.7 esperado:0 resultado:0

2613.4 11.0 3284.7 72.2 esperado:0 resultado:0

1.0 3.7 84.7 78.5 esperado:0 resultado:0

491.4 5.7 542.1 69.1 esperado:0 resultado:0

2814.8 10.7 3195.4 213.9 esperado:0 resultado:0

3603.0 14.4 3600.0 198.7 esperado:1 resultado:1

835.8 6.8 941.6 190.3 esperado:0 resultado:0

440.8 5.5 471.3 37.9 esperado:0 resultado:0

2374.5 10.1 2884.0 201.7 esperado:0 resultado:0

893.0 9.0 2194.3 26.7 esperado:0 resultado:0

In [ ]:
for recurso in recursado:
  valido = True

  while valido:
    val = input(recurso + ': ')

    if not val.isdigit(): valido = False

  prever.append(float(val))

previsoes = classificador.predict(entrada([prever]))

prever = []

print(f'Previsto para {funcionamento[list(previsoes[0]).index(max(list(previsoes[0])))]}')

LV-ActivePower-kW: 3603.0
Wind-Speed-m/s: 14.4
Theoretical-Power-Curve-KWh: 3600.0
Wind-Direction: 198.7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Previsto para Dentro do Limite Aceitável
